# Imports and Setup

## Asynchronous Execution

In [1]:
import threading

In [2]:
# multithreading lock to pause experiment loop during user feedback collection
experiment_lock = threading.Event()
experiment_lock.set()

## Graphical User Interface

In [3]:
import ipywidgets as ipw
import matplotlib.pyplot as plt
import ternary

# enable ipympl
%matplotlib widget

# turn on interactive mode so that plots only appear where we want it to
plt.ioff()

In [4]:
# create the gui elements
fig, tax = ternary.figure()

pause_button = ipw.Button(description='Pause')
undo_button = ipw.Button(description='Undo Point')
continue_button = ipw.Button(description='Continue')

output = ipw.Output()

In [5]:
# store a list of points where the user has clicked
points = []

In [6]:
def draw_background_plot(tax):
    '''
    Plot the data from the current experiment loop.

    Parameters
    ----------
    tax: ternary axes on which to plot the data
    '''

    pass

In [7]:
def draw_user_points(ax):
    '''
    Draw the points where the user has clicked.

    Parameters
    ----------
    ax: the axes on which to draw the points
    '''

    # create separate x and y coordinate lists for every point in points
    x, y = zip(*points)

    ax.plot(x, y, 'k*-')

    plt.draw()

In [8]:
def plot_callback(event):
    '''
    Store the point where the user clicked and add it to the plot.

    Parameters
    ----------
    event: a structure which has information about the mouse event
    '''
    
    if not experiment_lock.is_set():
        # add the mouse position (in data coordinates) to the points list
        points.append([event.xdata, event.ydata])

        draw_user_points(tax.get_axes())

cid = fig.canvas.mpl_connect('button_press_event', plot_callback)

In [9]:
def pause_callback(button):
    '''
    Set the internal flag of the threading Event object to false.

    Parameters
    ----------
    button: the instance of button that was clicked
    '''
    
    if experiment_lock.is_set():
        output.append_stdout('user has paused')
        experiment_lock.clear()

pause_button.on_click(pause_callback)

In [10]:
def continue_callback(button):
    '''
    Set the internal flag of the threading Event object to true

    Parameters
    ----------
    button: the instance of button that was clicked
    '''
    
    if not experiment_lock.is_set():
        output.append_stdout('user has unpaused')
        experiment_lock.set()

continue_button.on_click(continue_callback)

In [11]:
# assemble the gui elements
buttons = ipw.HBox([pause_button, undo_button, continue_button])
gui = ipw.VBox([fig.canvas, buttons, output])

## CAMEO

In [12]:
# get the simulation test data
# !wget -O CAMEO2_support_files_220104a.zip https://drive.google.com/u/0/uc?id=1UnKijzN_6shj-T2r37Jm2V6PSqTJMO-6&export=download
# !unzip -o CAMEO2_support_files_220104a.zip

In [27]:
import numpy as np
import gpflow
from gpflow.ci_utils import ci_niter
from sklearn.manifold import spectral_embedding
from sklearn.mixture import GaussianMixture
from scipy.spatial import Voronoi
from scipy.spatial.distance import pdist, squareform
from ternary.helpers import project_sequence
from scipy.io import loadmat
from scipy.stats import entropy

In [14]:
def phase_mapping(X, num_clusters):
    '''
    Cluster data using spectral clustering and a Gaussian mixture model

    Parameters
    ----------
    X: m x n matrix - m rows of n dimensional data
    num_clusters: number of groups to cluster data into

    Returns
    -------
    cl: clustering labels for each sample
    cluster_prob: the probability for each sample to belong to each cluster
    '''

    K = similarity_matrix(X, 'cosine')

    if X.shape[0] <= num_clusters:
        # fewer data points than clusters, each point gets its own cluster
        cluster_prob = np.eye(X.shape[0])
    else:
        x_se = spectral_embedding(adjacency=K, n_components=num_clusters)
        model = GaussianMixture(n_components=num_clusters,
                                covariance_type='diag',
                                n_init=100).fit(x_se)

        cluster_prob = model.predict_proba(x_se)

    cl = np.argmax(cluster_prob, axis=1).flatten()
    return cl, cluster_prob

In [15]:
def composition_to_graph(T):
    '''
    Use the composition data to identify nearby neighbors graph in
    composition-space.

    Parameters
    ----------
    T: n x 3 matrix - n rows of composition data in ternary coordinate space

    Returns
    -------
    S: n x n matrix - symmetric adjacency matrix
    '''

    N = T.shape[0]
    XYc = np.array(list(zip(*project_sequence(T))))
    vor = Voronoi(XYc)
    points_separated = vor.ridge_points
    S = np.zeros((N, N))
    for i in range(points_separated.shape[0]):
        S[points_separated[i, 0], points_separated[i, 1]] = 1
        S[points_separated[i, 1], points_separated[i, 0]] = 1
    return S

In [16]:
def plot_graph(S, XY):
    '''
    Plot the graph learned from composition positions.

    Parameters
    ----------
    S: n x n matrix - symmetric adjacency matrix
    XY: n x 2 matrix - n rows of composition data in cartesian coordinates
    '''

    r, c = np.nonzero(S)
    for i in range(r.shape[0]):
        xx = [XY[r[i], 0], XY[c[i], 0]]
        yy = [XY[r[i], 1], XY[c[i], 1]]
        plt.plot(xx, yy, c=np.asarray([.8, .8, .8]))
        plt.plot(xx, yy, 'k.')

In [17]:
def prune_graph(S, XY, dist_ratio, num_nearest_neighbors):
    '''
    Remove edges from the graph that connect points which are too far away.

    Parameters
    ----------
    S: n x n matrix - symmetric adjacency matrix
    XY: n x 2 matrix - n rows of composition data in cartesian coordinates
    dist_ratio: adjusts the cutoff distance
    num_nearest_neighbors: desired degree for data vertices
    '''

    D = squareform(pdist(XY))
    mD = np.sort(D, 0)
    mD = mD[num_nearest_neighbors, :]
    S_ = S.copy()
    for i in range(S.shape[0]):
        d = D[i, :]
        S_[i, d > dist_ratio*mD[i]] = 0
        S_[d > dist_ratio*mD[i], i] = 0
    return S_

In [18]:
def similarity_matrix(X, metric, sigma=1):
    '''
    Calculate and return the similarity matrix used in spectral clustering.

    Parameters
    ----------
    X: m x n matrix - m rows of n dimensional data
    metric: distance metric passed to scipy.spatial.distance.pdist()
    sigma: scaling factor for Gaussian radial basis function (default=1)
    '''

    distance = squareform(pdist(X, metric))
    W = np.exp(-(distance**2) / (2*sigma**2))
    return W

In [28]:
def gpc_phasemapping(xy_curr, labels_curr, xy_full, num_clusters,
                     weight_prior=None):
    '''
    Take clustering labels for the samples and then extrapolate them throughout
    composition space, segmenting the XY space into 'phase regions'.

    Parameters
    ----------
    xy_curr: cartesian coordinates of measured data points
    labels_curr: cluster labels for those data
    xy_full: cartesian coordinates of measured and new, query data points
    num_clusters: the number of clusters we're assuming exist
    weight_prior: variance coefficient of (optional) prior kernel

    Returns
    -------
    y_mean: data point label predictions
    y_variance: data point label variances
    f_mean: data point latent GP predictions
    f_variance: data point latent GP variances
    '''

    data = (xy_curr, labels_curr)

    if weight_prior is None:
        kernel = gpflow.kernels.Matern32(lengthscales=[1, 1])
    else:
        prior_kernel = gpflow.kernels.SquaredExponential(active_dims=[2],
                                                         lengthscales=0.001,
                                                         variance=weight_prior)

        # fix the prior kernel lengthscale and variance
        gpflow.utilities.set_trainable(prior_kernel.parameters[1], False)
        gpflow.utilities.set_trainable(prior_kernel.parameters[0], False)
                                                         
        kernel = gpflow.kernels.Matern32(active_dims=[0, 1]) + prior_kernel

    invlink = gpflow.likelihoods.RobustMax(num_clusters)

    likelihood = gpflow.likelihoods.MultiClass(num_clusters, invlink=invlink)

    model = gpflow.models.VGP(data=data,
                              kernel=kernel,
                              likelihood=likelihood,
                              num_latent_gps=num_clusters)

    # hyperparameter optimization
    opt = gpflow.optimizers.Scipy()
    _ = opt.minimize(model.training_loss,
                     model.trainable_variables,
                     options={'maxiter': ci_niter(1000)})

    # Poisson process for the full XY coordinates
    y = model.predict_y(xy_full)
    y_mean = y[0].numpy()
    y_variance = y[1].numpy()

    # (non-squeezed) probabilistic function for class labels
    f = model.predict_f(xy_full)
    f_mean = f[0].numpy()
    f_variance = f[1].numpy()

    return y_mean, y_variance, f_mean, f_variance

## Experiment Loop

In [33]:
def main(output):
    '''
    Run the calculation loop while checking if the user is waiting to
    provide input. If so, pause the loop, collect input, then resume.

    Parameters
    ----------
    output: the ipywidgets Output widget to print messages
    '''
    output.append_stdout('starting experiment\n')

    data = loadmat('FeGaPd_full_data_220104a.mat')

    # composition data in cartesian coordinates
    composition = data['C']
    idx = [1, 2, 0]
    cartesian = np.array(list(zip(*project_sequence(composition[:, idx]))))

    # x ray diffraction data
    xrd = data['X'][:, 631:1181]

    num_measurements = 20
    num_clusters = 5
    num_total = cartesian.shape[0]

    measured = np.random.permutation(num_total)[:num_measurements]

    for i in range(10):
        labels, _ = phase_mapping(xrd[measured, :], num_clusters)

        y_mean, _, _, _ = gpc_phasemapping(cartesian[measured, :],
                                           labels,
                                           cartesian,
                                           num_clusters)

        acquisition = entropy(y_mean, axis=1)

        next_sample = np.argmax(acquisition)

        measured = np.append(measured, next_sample)

        output.append_stdout(f'iteration {i} chose sample {next_sample}\n')

        if not experiment_lock.is_set():
            experiment_lock.wait()

    output.append_stdout('all done\n')


# Putting It Together

In [34]:
# begin the experiment in a new thread
output.clear_output()
experiment_thread = threading.Thread(target=main,
                                     args=(output,),
                                     name='experiment')
experiment_thread.start()

In [35]:
# start the gui
gui

In [32]:
experiment_thread.is_alive()

False